In [ ]:
# https://robbiegeoghegan.medium.com/implementing-a-b-tests-in-python-514e9eb5b3a1

### AB test with Python
A data from kaggle: https://www.kaggle.com/zhangluyuan/ab-testing

In [1]:
# suppose we have load the data as df

In [ ]:
# remove some wrong records:
mask = ((df["group"] == "control") & (df["landing_page"] == "new_page")) | ((df["group"] == "treatment") & (df["landing_page"] == "old_page"))
ix_to_drop = df[mask].index
df = df.drop(ix_to_drop)

In [ ]:
# Metric is conversion rate
# H_0: difference = 0; H_1: difference != 0

In [ ]:
mask = (df["group"] == "control")
conversions_control = df["converted"][mask].sum()
total_users_control = df["converted"][mask].count()

mask = (df["group"] == "treatment")
conversions_treatment = df["converted"][mask].sum()
total_users_treatment = df["converted"][mask].count()

In [ ]:
practical_sig = 0.01
statistical_sig = 0.05
power = 0.8
baseline_rate = conversions_control/total_control

def cal_sample_size(sig, power, p, practical_sig):
    sigma_square = p*(1-p)
    return 16*sigma_square/(practical_sig**2)

sample_size = cal_sample_size(statistical_sig,
                             power,
                             baseline_rate,
                             practical_sig)
print("required sample size per group:", round(sample_size))    

In [ ]:
# calculate pooled rate to calculate the standard error
prob_pooled = (conversions_control+conversions_treatment)/(total_users_control+total_users_treatment)
se_pooled = math.sqrt(prob_pooled * (1-prob_pooled) * (1/total_users_control + 1/total_users_treatement))
z_score = st.norm.ppf(1-statistical_sig)
margin = se_pooled * z_score

d_hat = conversions_treatment/total_users_treatment - conversions_control/total_users_control

lower, upper = d_hat - margin, d_hat + margin

if practical_sig < lower:
    print("we should reject the null hypothesis")
else:
    print("can not reject the null hypothesis")

In [ ]:
# Another kernel: https://www.kaggle.com/tammyrotem/ab-tests-with-python